# Run your first Qiskit Serverless workload remotely



<details>
  <summary><b>Package versions</b></summary>

  The code on this page was developed using the following requirements.
  We recommend using these versions or newer.

  ```
  qiskit[all]~=1.3.1
  qiskit-ibm-runtime~=0.34.0
  qiskit-aer~=0.15.1
  qiskit-serverless~=0.18.1
  qiskit-ibm-catalog~=0.2
  qiskit-addon-sqd~=0.8.1
  qiskit-addon-utils~=0.1.0
  qiskit-addon-mpf~=0.2.0
  qiskit-addon-aqc-tensor~=0.1.2
  qiskit-addon-obp~=0.1.0
  scipy~=1.15.0
  pyscf~=2.8.0
  ```
</details>



This section explores how to use `qiskit-ibm-catalog` to list programs available in Qiskit Serverless, pass inputs into these programs, run them remotely, check their status, and retrieve results and logs.

Be sure you have authenticated to Qiskit Serverless with your [IBM Quantum account](https://quantum.ibm.com/account) (see [Deploy to IBM Quantum Platform](./serverless-first-program#deploy-to-ibm-quantum-platform) for instructions).



## List programs available

You can use `QiskitServerless.list()` to fetch a list of the available programs to run with Qiskit Serverless. This includes the previously uploaded [`transpile_remote_serverless`](./serverless-first-program).



In [1]:
from qiskit_ibm_catalog import QiskitServerless

serverless = QiskitServerless()
next(
    program
    for program in serverless.list()
    if program.title == "transpile_remote_serverless"
)

QiskitFunction(transpile_remote_serverless)

## Run an uploaded program and pass inputs

First, set up your inputs. Your program has three inputs: `circuits`, `backend`, and `optimization_level`. You can use `random_circuit` to create 30 random circuits:



In [2]:
from qiskit.circuit.random import random_circuit

qc_random = [(random_circuit(4, 4, measure=True, seed=i)) for i in range(10)]
qc_random[0].draw(output="mpl", idle_wires=False)

<Image src="/images/extracted-notebook-images/serverless-run-first-workload/46108a95-11f8-4ecf-83bb-131c3e7ea4c2-0.svg" alt="Output from the previous code" />

Next, use `QiskitRuntimeService` and `least_busy` to select a `backend`:



In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

# If you have not previously saved your credentials, follow instructions at
# https://docs.quantum.ibm.com/guides/setup-channel#iqp
# to authenticate with your API token.
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)

ibm_kyiv


Set your optimization level:



In [4]:
optimization_level = 3

Select your program with `serverless.load('PROGRAM_NAME')`:



In [5]:
transpile_remote_serverless = serverless.load("transpile_remote_serverless")

Next, pass your inputs and run it in a pythonic fashion as follows:



In [6]:
job = transpile_remote_serverless.run(
    circuits=qc_random,
    backend=backend.name,
    optimization_level=optimization_level,
)

In [7]:
job.job_id

'1bb26ca6-7406-47c3-9d56-a3d0757907af'

## Check job status

With your Qiskit Serverless `job_id`, you can check the status of running jobs. This includes the following statuses:

*   **`QUEUED`**: The remote program is in the Qiskit Serverless queue. The queue priority is currently based on how much you've used Qiskit Serverless
*   **`INITIALIZING`**: The remote program is starting; this includes setting up the remote environment and installing dependencies
*   **`RUNNING`**: The program is running. At this stage, if you have `print()` outputs in your program, you can retrieve logs using `job.logs()`
*   **`DONE`**: The program is complete, and you can retrieve data stored in `save_result()` with `job.results()`



In [8]:
job.status()

'QUEUED'

<Admonition type="tip">
  Currently, the IBM Quantum workloads table only reflects Qiskit Runtime workloads. Use `job.status()` to see your Qiskit Serverless workload's current status.
</Admonition>

You've successfully run your first Qiskit Serverless program!



## Retrieve logs and results

As mentioned before, once a program is `RUNNING`, you can use `job.logs()` to fetch logs created from `print()` outputs:



In [10]:
logs = job.logs()
print(logs)

No logs yet.


At any time, you can also cancel a job:



In [11]:
job.stop()

'Job has been stopped.'

Once a program is `DONE`, you can use `job.results()` to fetch the result stored in `save_result()`:



In [12]:
# We can't get results from a cancelled job, so we'll fetch a completed one instead
completed_job = next(
    job for job in serverless.jobs() if job.status() == "DONE"
)
completed_job.result()

## List previously run jobs run with Qiskit Serverless

You can use `jobs()` to list all jobs submitted to Qiskit Serverless:



In [13]:
old_jobs = serverless.jobs()
old_jobs

[<Job | 1bb26ca6-7406-47c3-9d56-a3d0757907af>,
 <Job | d0b42704-7ca2-4220-aa76-6fed54e37ea9>,
 <Job | fcfad8c6-2a5a-4dd8-a4d8-5089d3164ff4>,
 <Job | 4e238d0a-9c07-4b18-924b-bb3b8254e6a7>,
 <Job | 5103fa3d-10cb-4398-a165-2144f17780c3>,
 <Job | 0741eba2-60b5-4929-b4c4-09f7cc8f1613>,
 <Job | 394d0e47-2933-4067-a89e-e0df370da5d7>,
 <Job | 43fc6a0e-1d6b-4054-bf49-80bea7273405>,
 <Job | 222f0724-5b12-412c-a8c5-4fac0ab0d9fa>,
 <Job | cbbe84f7-6f1a-41ac-bac5-c1cfebeb7dde>]

## Next steps

<Admonition type="info" title="Recommendations">
  *   Explore [compute and data management tools](./serverless-manage-resources) available to your program, including parallelization.
</Admonition>

